In [1]:
import librosa as lb
import os, glob, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from librosa.display import specshow, waveplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import utils
from tensorflow import math
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, plot_confusion_matrix, confusion_matrix
from sklearn.preprocessing import scale

import autoreload
%load_ext autoreload
%autoreload 2

import scripts.sound_feature_extraction as sfe
import scripts.common_quick_plots as qp

np.random.seed(9) #For the Mighty Nein!

## Phase 4: Modeling

### 4.1 Modeling Overview

#### Sound Feature Transformation

At current, the community of data scientists working on speech emotion recognition began using feature transformed sound similar to the digitization techniques used by major telecommunications and  paired with shallow neural networks.  The most common transformation used directly on sound data is is the MFCC, or Mel-Frequency Cepstrum Coefficients.  

Per Wikipedia, a "mel-frequency cepstrum (MFC) is a representation of the shor term power spectrum of a sound, based on a linear cosine transform of a log power spectrum on a nonlinear mel scale of frequency." https://en.wikipedia.org/wiki/Mel-frequency_cepstrum

If that confuses you, its okay.  There are a lot of different pieces to unpack, and if you're interested in learning more about the process, I highly recommend checking out Jack Schaedler's visual explanation of the steps involved in digital signal processing-- https://jackschaedler.github.io/circles-sines-signals/index.html 
It's a good read and well worth your time, although the entire article is a deep dive into the first of 5 steps to acquire the MFCC's:

1. take a Fourier transform of the sample
2. map the results onto the mel scale
3. log the powers at each mel frequency
4. take a discrete cosine transform of each item of the mel log powers
5. sample the amplitudes of the resulting spectrum.

Fortunately, this was standardized by the telecommunications industry and is easily achieved using the librosa sound processing library.  Take a look at the scripts/sound_feature_extraction.py file for more details on how it was accomplished in this example.

#### Spectrogram Image Transformation

More recently, data scientists have been using image recognition techniques with deep learning as an alternative method, and convolutional neural networks trained on sound image data (mostly spectrograms) have seen an increase in effectiveness as a result.

As recently as March of 2020, the debate between the two primary methods has been ongoing-- 
https://www.frontiersin.org/articles/10.3389/fcomp.2020.00014/full#F3

### 4.2 Speech and Song Side by Side

The first model I want to look at is one that attempts to classify the emotions regardless of 

In [2]:
tar_emotions = list(sfe.emotions.values())
tar_emotions.remove('neutral')
# removing neutral emotions as 'neutral' and 'calm' are too similar

tar_actors = ['01', '02', '03', '04', '05', '06', '07',
              '08', '09', '10', '11', '12', '13', '14',
              '15','16', '17', '18', '19', '20', '21', '22', '23', '24'] 
# Using all actors

In [ ]:
df = sfe.load_targets(tar_emotions, tar_actors)

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:174: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:191: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:174: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/preprocess

In [ ]:
df['feature_length'] = [len(feature) for feature in df['flat_feature']]
df['feature_length'].value_counts()

In [ ]:
df['mfcc_shape'] = [mfcc.shape for mfcc in df['mfccs']]
df['mfcc_shape'].value_counts()

In [ ]:
df['melspec_shape'] = [melspec.shape for melspec in df['melspec']]
df['melspec_shape'].value_counts()

In [ ]:
df['feature_array_shape'] = [feature.shape for feature in df['feature_array']]
df['feature_array_shape'].value_counts()

In [ ]:
split_emotions = ['happy', 'sad', 'angry', 'fearful']
sp_so_df = df[df['emotion'].isin(split_emotions)].copy()

In [ ]:
X = np.stack(sp_so_df['flat_feature'].to_numpy())
y = sp_so_df['emotion'].copy()
y = pd.get_dummies(y, columns = ['emotion'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [ ]:
n_input = X_train[0].shape

In [ ]:
model = Sequential()
model.add(Dense(512,
               activation = 'relu',
               input_shape = n_input))
model.add(Dropout(0.25))
model.add(Dense(256,activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(128,activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(64,activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(32,activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(16, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(4, activation = 'softmax'))
          
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])
res = model.fit(X_train, y_train,
               validation_data = (X_test, y_test),
               epochs = 250,
               verbose = 0)

In [ ]:
qp.plot_loss(res, 'Model Training and Testing Loss by Epoch')

In [ ]:
qp.plot_accuracy(res, 'Model Training and Testing Loss by Epoch')

In [ ]:
qp.plot_confusion_matrix(model, X_test, y_test, "Confusion Matrix of Classified Emotions")

In [ ]:
res.history['accuracy'][-1], res.history['val_accuracy'][-1]

In [ ]:
X = np.stack(df['flat_feature'].to_numpy())
y = df['emotion'].copy()
y = pd.get_dummies(y, columns = ['emotion'])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
n_input = X_train[0].shape

In [ ]:
flat_model = Sequential()
flat_model.add(Dense(512,
               activation = 'relu',
               input_shape = n_input))
flat_model.add(Dropout(0.25))
flat_model.add(Dense(256,activation = 'relu'))
flat_model.add(Dropout(0.25))
flat_model.add(Dense(128,activation = 'relu'))
flat_model.add(Dropout(0.25))
flat_model.add(Dense(64,activation = 'relu'))
flat_model.add(Dropout(0.25))
flat_model.add(Dense(32,activation = 'relu'))
flat_model.add(Dropout(0.25))
flat_model.add(Dense(16, activation = 'relu'))
flat_model.add(Dropout(0.25))
flat_model.add(Dense(7, activation = 'softmax'))
          
flat_model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])
flat_res = flat_model.fit(X_train, y_train,
               validation_data = (X_test, y_test),
               epochs = 250,
               verbose = 0)

In [ ]:
qp.plot_loss(flat_res, 'Flat Feature Model Training and Testing Loss by Epoch')

In [ ]:
qp.plot_accuracy(flat_res, 'Flat Feature Model Training and Testing Accuracy by Epoch')

In [ ]:
qp.plot_confusion_matrix(flat_model, X_test, y_test, 'Confusion Matrix of Classified Emotions- Flat Features Model')

In [ ]:
flat_res.history['accuracy'][-1], flat_res.history['val_accuracy'][-1]

In [ ]:
speech_tar_emotions = ['happy','sad','angry','fearful', 'disgust','surprised']
speech_df = df[(df['channel'] == 'speech') & (df['emotion'].isin(speech_tar_emotions))]

In [ ]:
X = np.stack(speech_df['flat_feature'].to_numpy())
y = speech_df['emotion'].copy()
y = pd.get_dummies(y, columns = ['emotion'])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [ ]:
sp_model = Sequential()
sp_model.add(Dense(512,
               activation = 'relu',
               input_shape = n_input))
sp_model.add(Dropout(0.25))
sp_model.add(Dense(256,activation = 'relu'))
sp_model.add(Dropout(0.25))
sp_model.add(Dense(128,activation = 'relu'))
sp_model.add(Dropout(0.25))
sp_model.add(Dense(64,activation = 'relu'))
sp_model.add(Dropout(0.25))
sp_model.add(Dense(32,activation = 'relu'))
sp_model.add(Dropout(0.25))
sp_model.add(Dense(16, activation = 'relu'))
sp_model.add(Dropout(0.25))
sp_model.add(Dense(6, activation = 'softmax'))
          
sp_model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [ ]:
sp_res = sp_model.fit(X_train, y_train,
               validation_data = (X_test, y_test),
               epochs = 250,
               verbose = 0)

In [ ]:
qp.plot_loss(sp_res, 'Model Training and Testing Loss by Epoch- Speech Files Only')

In [ ]:
qp.plot_accuracy(sp_res, 'Model Training and Testing Accuracy by Epoch- Speech Files Only')

In [ ]:
plot_confusion_matrix(sp_model, X_test, y_test, "Confusion Matrix of Classified Emotions- Speech Files Only")

In [ ]:
sp_res.history['accuracy'][-1], sp_res.history['val_accuracy'][-1]

In [ ]:
song_tar_emotions = ['happy','sad','angry','fearful', 'disgust','surprised']
song_df = df[(df['channel'] == 'song') & (df['emotion'].isin(song_tar_emotions))]

In [ ]:
X = np.stack(song_df['flat_feature'].to_numpy())
y = song_df['emotion'].copy()
y = pd.get_dummies(y, columns = ['emotion'])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [ ]:
n_input = X_train[0].shape
n_input

In [ ]:
so_model = Sequential()
so_model.add(Dense(512,
               activation = 'relu',
               input_shape = n_input))
so_model.add(Dropout(0.25))
so_model.add(Dense(256,activation = 'relu'))
so_model.add(Dropout(0.25))
so_model.add(Dense(128,activation = 'relu'))
so_model.add(Dropout(0.25))
so_model.add(Dense(64,activation = 'relu'))
so_model.add(Dropout(0.25))
so_model.add(Dense(32,activation = 'relu'))
so_model.add(Dropout(0.25))
so_model.add(Dense(16, activation = 'relu'))
so_model.add(Dropout(0.25))
so_model.add(Dense(4, activation = 'softmax'))
          
so_model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

In [ ]:
so_res = so_model.fit(X_train, y_train,
               validation_data = (X_test, y_test),
               epochs = 250,
               verbose = 0)

In [ ]:
qp.plot_accuracy(so_res, 'Model Training and Testing Accuracy by Epoch- Song Files Only')

In [ ]:
qp.plot_loss(so_res, 'Model Training and Testing Loss by Epoch- Song Files Only')

In [ ]:
qp.plot_confusion_matrix(so_model, X_test, y_test, "Confusion Matrix of Classified Emotions- Song Files Only")

In [ ]:
so_res.history['accuracy'][-1], so_res.history['val_accuracy'][-1]

## CNN- Convolutional Neural Network using MFCC's

In [ ]:
X = np.stack(df['mfccs'])
y = df['emotion'].copy()
y = pd.get_dummies(y, columns = ['emotion'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
X_train.shape, X_test.shape

In [ ]:
Z_train = X_train.reshape(-1, 40, 275, 1)
Z_test = X_test.reshape(-1, 40, 275, 1)

In [ ]:
mfcc_cnn = Sequential()
mfcc_cnn.add(Conv2D(filters = 32,
                    kernel_size = (3,3),
                    activation = 'relu',
                    input_shape = (40, 275, 1)))
mfcc_cnn.add(MaxPooling2D(pool_size = (3,2)))
mfcc_cnn.add(Flatten())
mfcc_cnn.add(Dense(128, activation = 'relu'))
mfcc_cnn.add(Dropout(0.25))
mfcc_cnn.add(Dense(64, activation = 'relu'))
mfcc_cnn.add(Dropout(0.25))
mfcc_cnn.add(Dense(32, activation = 'relu'))
mfcc_cnn.add(Dense(7, activation = 'softmax'))
mfcc_cnn.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])
early_stop = EarlyStopping(patience = 5)

In [ ]:
mfcc_cres = mfcc_cnn.fit(Z_train, y_train,
               batch_size = 112,
               validation_data = (Z_test, y_test),
               epochs = 50,
               callbacks = [early_stop],
               verbose = 0)

In [ ]:
mfcc_cres.history['accuracy'][-1], mfcc_cres.history['val_accuracy'][-1]

In [ ]:
qp.plot_loss(mfcc_cres, 'CNN Model Training and Testing Loss by Epoch')

In [ ]:
qp.plot_accuracy(mfcc_cres, 'CNN Model Training and Testing Accuracy by Epoch')

In [ ]:
qp.plot_confusion_matrix(mfcc_cnn, Z_test, y_test, 'CNN Model Testing Confusion Matrix')

We're actually looking at a decrease in accuracy when we use just the MFCC array in a convolutional neural network.  

In [ ]:
df['melspec'][1].shape

In [ ]:
df['melspec_len'] = [mel.shape[-1] for mel in df['melspec']]
df['melspec_len'].value_counts()

In [ ]:
X = np.stack(df['melspec'])
y = df['emotion'].copy()
y = pd.get_dummies(y, columns = ['emotion'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
X_train.shape, X_test.shape

In [ ]:
Z_train = X_train.reshape(-1, 128, 275, 1)
Z_test = X_test.reshape(-1, 128, 275, 1)

In [ ]:
Z_train.shape, Z_test.shape

In [ ]:
mel_cnn = Sequential()
mel_cnn.add(Conv2D(filters = 16,
                    kernel_size = (3,3),
                    activation = 'relu',
                    input_shape = (128, 275, 1)))
mel_cnn.add(MaxPooling2D(pool_size = (3,2)))
mel_cnn.add(Conv2D(filters = 16,
                    kernel_size = (3,3),
                    activation = 'relu'))
mel_cnn.add(MaxPooling2D(pool_size = (3,2)))
mel_cnn.add(Conv2D(filters = 16,
                    kernel_size = (3,3),
                    activation = 'relu'))
mel_cnn.add(MaxPooling2D(pool_size = (3,2)))
mel_cnn.add(Flatten())
mel_cnn.add(Dense(128, activation = 'relu'))
mel_cnn.add(Dropout(0.25))
mel_cnn.add(Dense(64, activation = 'relu'))
mel_cnn.add(Dropout(0.25))
mel_cnn.add(Dense(32, activation = 'relu'))
mel_cnn.add(Dense(7, activation = 'softmax'))
mel_cnn.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])
early_stop = EarlyStopping(patience = 5)
mel_res = mel_cnn.fit(Z_train, y_train,
               batch_size = 256,
               validation_data = (Z_test, y_test),
               epochs = 50,
               callbacks = [early_stop],
               verbose = 0)

In [ ]:
mel_res.history['accuracy'][-1], mel_res.history['val_accuracy'][-1]

In [ ]:
qp.plot_loss(mel_res, 'Melspectrogram CNN Training and Validation Loss by Epoch')

In [ ]:
qp.plot_accuracy(mel_res, 'Melspectrogram CNN Training and Validation Accuracy by Epoch')

In [ ]:
qp.plot_confusion_matrix(mel_cnn, Z_test, y_test, 'Melspectrogram CNN Confusion Matrix')

### Full Feature Array CNN

In [ ]:
df['full_feature_len'] = [mel.shape[-1] for mel in df['feature_array']]
df['full_feature_len'].value_counts()

In [ ]:
X = np.stack(df['feature_array'])
y = df['emotion'].copy()
y = pd.get_dummies(y, columns = ['emotion'])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
X_train.shape, X_test.shape

In [ ]:
Z_train = X_train.reshape(-1, 180, 275, 1)
Z_test = X_test.reshape(-1, 180, 275, 1)
Z_train.shape, Z_test.shape

In [ ]:
ff_cnn = Sequential()
ff_cnn.add(Conv2D(filters = 16,
                    kernel_size = (5,5),
                    activation = 'relu',
                    input_shape = (180, 275, 1)))
ff_cnn.add(MaxPooling2D(pool_size = (2,2)))
ff_cnn.add(Conv2D(filters = 16,
                    kernel_size = (4,4),
                    activation = 'relu'))
ff_cnn.add(MaxPooling2D(pool_size = (2,2)))
ff_cnn.add(Conv2D(filters = 16,
                    kernel_size = (3,3),
                    activation = 'relu'))
ff_cnn.add(MaxPooling2D(pool_size = (2,2)))
ff_cnn.add(Conv2D(filters = 16,
                    kernel_size = (2,2),
                    activation = 'relu'))
ff_cnn.add(MaxPooling2D(pool_size = (2,2)))
ff_cnn.add(Flatten())
ff_cnn.add(Dense(128, activation = 'relu'))
ff_cnn.add(Dropout(0.33))
ff_cnn.add(Dense(64, activation = 'relu'))
ff_cnn.add(Dropout(0.33))
ff_cnn.add(Dense(32, activation = 'relu'))
ff_cnn.add(Dense(7, activation = 'softmax'))
ff_cnn.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])
early_stop = EarlyStopping(patience = 5)
ff_cres = ff_cnn.fit(Z_train, y_train,
               batch_size = 256,
               validation_data = (Z_test, y_test),
               epochs = 100,
               callbacks = [early_stop],
               verbose = 0)

In [ ]:
ff_cres.history['accuracy'][-1], ff_cres.history['val_accuracy'][-1]

In [ ]:
qp.plot_loss(ff_cres, 'Full Feature CNN Training and Validation Loss by Epoch')

In [ ]:
qp.plot_accuracy(ff_cres, 'Full Feature CNN Training and Validation Accuracy by Epoch')

In [ ]:
qp.plot_confusion_matrix(ff_cnn, Z_test, y_test, 'Full Feature CNN Confusion Matrix')

In [ ]:
X = np.stack(sp_so_df['feature_array'])
y = sp_so_df['emotion'].copy()
y = pd.get_dummies(y, columns = ['emotion'])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
X_train.shape, X_test.shape

In [ ]:
Z_train = X_train.reshape(-1, 180, 275, 1)
Z_test = X_test.reshape(-1, 180, 275, 1)
Z_train.shape, Z_test.shape

In [ ]:
spso_ff_cnn = Sequential()
spso_ff_cnn.add(Conv2D(filters = 16,
                    kernel_size = (4,4),
                    activation = 'relu',
                    input_shape = (180, 275, 1)))
spso_ff_cnn.add(MaxPooling2D(pool_size = (3,2)))
spso_ff_cnn.add(Conv2D(filters = 16,
                    kernel_size = (3,3),
                    activation = 'relu'))
spso_ff_cnn.add(MaxPooling2D(pool_size = (3,2)))
spso_ff_cnn.add(Conv2D(filters = 16,
                    kernel_size = (2,2),
                    activation = 'relu'))
spso_ff_cnn.add(MaxPooling2D(pool_size = (2,2)))
spso_ff_cnn.add(Flatten())
spso_ff_cnn.add(Dense(128, activation = 'relu'))
spso_ff_cnn.add(Dropout(0.33))
spso_ff_cnn.add(Dense(64, activation = 'relu'))
spso_ff_cnn.add(Dropout(0.33))
spso_ff_cnn.add(Dense(32, activation = 'relu'))
spso_ff_cnn.add(Dense(4, activation = 'softmax'))
spso_ff_cnn.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])
early_stop = EarlyStopping(patience = 4)
spso_ff_cres = spso_ff_cnn.fit(Z_train, y_train,
               batch_size = 256,
               validation_data = (Z_test, y_test),
               epochs = 100,
               callbacks = [early_stop],
               verbose = 0)

In [ ]:
round(spso_ff_cres.history['accuracy'][-1], 2), round(spso_ff_cres.history['val_accuracy'][-1], 2)

In [ ]:
qp.plot_loss(spso_ff_cres, 'Full Feature CNN Training and Validation Loss by Epoch- Speech Song Intersection Only')

In [ ]:
qp.plot_accuracy(spso_ff_cres, 'Full Feature CNN Training and Validation Accuracy by Epoch-- Speech Song Intersection Only')

In [ ]:
qp.plot_confusion_matrix(spso_ff_cnn, Z_test, y_test, 'Full Feature CNN Confusion Matrix- Speech Song Intersection Only')

In [ ]:
accuracy_summary = {
    'Speech Only Full Features Flattened' : [round(sp_res.history['accuracy'][-1], 2), round(sp_res.history['val_accuracy'][-1], 2)],
    'Songs Only Full Features Flattened'  : [round(so_res.history['accuracy'][-1], 2), round(so_res.history['val_accuracy'][-1], 2)],
    'Speech-Song Intersection- Full Feature Flattened'              : [round(res.history['accuracy'][-1], 2), round(res.history['val_accuracy'][-1], 2)],
    'Full Feature Flattened'              : [round(flat_res.history['accuracy'][-1], 2), round(flat_res.history['val_accuracy'][-1], 2)],
    'MFCC CNN'                            : [round(mfcc_cres.history['accuracy'][-1], 2), round(mfcc_cres.history['val_accuracy'][-1], 2)],
    'Melspectrogram CNN'                  : [round(mel_res.history['accuracy'][-1], 2), round(mel_res.history['val_accuracy'][-1], 2)],
    'Speech-Song Intersection- Full Feature CNN' : [round(spso_ff_cres.history['accuracy'][-1], 2), round(spso_ff_cres.history['val_accuracy'][-1], 2)],
    'Full Feature CNN'                    : [round(ff_cres.history['accuracy'][-1], 2), round(ff_cres.history['val_accuracy'][-1], 2)]
}
accuracy_df = pd.DataFrame.from_dict(accuracy_summary, orient='index',
                       columns=['Train', 'Test'])
accuracy_df['Variance'] = abs(accuracy_df['Train'] - accuracy_df['Test'])
accuracy_df